In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score


In [2]:
# Load the dataset and check the first few rows
all_season_details_path = 'C:/Users/Lenovo/Downloads/all_season_details.csv'
all_season_details_df = pd.read_csv(all_season_details_path)

all_season_details_df.head()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19172\1973567284.py:3: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  all_season_details_df = pd.read_csv(all_season_details_path)


,comment_id,season,match_id,match_name,home_team,away_team,current_innings,innings_id,over,ball,...,wicket_id,wkt_batsman_name,wkt_bowler_name,wkt_batsman_runs,wkt_batsman_balls,wkt_text,isRetiredHurt,text,preText,postText
0,110,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"nice and full, angling and perhaps swinging in...",<p><strong>7.30pm</strong> Gaikwad and Conway ...,NaN
1,120,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"Conway shuffles across off, and shimmies out a...",NaN,NaN
2,130,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"good length, angling in at off and then straig...",NaN,NaN
3,140,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,False,"shorter and slanting into middle and leg, Gaik...","<p>Rummy: ""Fast bowling options for CSK defini...",NaN
4,150,2023.0,1359475,GT v CSK,GT,CSK,CSK,1,1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,False,Shami hits a heavy length and brings this back...,"<p>LG: ""Watching out for Hangargekar. He was i...",NaN


In [3]:
df=all_season_details_df

In [4]:
df.runs.max()

7

In [5]:
# Select relevant columns and process missing values
ball_by_ball_df = all_season_details_df[['season', 'match_id', 'home_team', 'away_team', 'current_innings', 'over', 'ball', 
                                         'wkt_batsman_runs', 'isRetiredHurt']].copy()

# Fill missing values in wkt_batsman_runs with 0
ball_by_ball_df['wkt_batsman_runs'] = ball_by_ball_df['wkt_batsman_runs'].fillna(0)

# Calculate Cumulative Runs for the innings
ball_by_ball_df['cumulative_runs'] = ball_by_ball_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs'].cumsum().reset_index(drop=True)

# Calculate Wickets Lost by counting the number of balls where a batsman was out
ball_by_ball_df['wickets_lost'] = ball_by_ball_df.groupby(['match_id', 'current_innings'])['wkt_batsman_runs'].apply(lambda x: (x > 0).cumsum()).reset_index(drop=True)

# Calculate Overs Bowled
ball_by_ball_df['overs_bowled'] = ball_by_ball_df['over'] + (ball_by_ball_df['ball'] / 6)

# Calculate Run Rate
ball_by_ball_df['run_rate'] = ball_by_ball_df['cumulative_runs'] / ball_by_ball_df['overs_bowled']

# Create Powerplay Indicator
ball_by_ball_df['powerplay'] = ball_by_ball_df['over'] < 6


In [6]:
ball_by_ball_df

,season,match_id,home_team,away_team,current_innings,over,ball,wkt_batsman_runs,isRetiredHurt,cumulative_runs,wickets_lost,overs_bowled,run_rate,powerplay
0,2023.0,1359475,GT,CSK,CSK,1,1,0.0,False,0.0,0,1.166667,0.000000,True
1,2023.0,1359475,GT,CSK,CSK,1,2,0.0,False,0.0,0,1.333333,0.000000,True
2,2023.0,1359475,GT,CSK,CSK,1,3,0.0,False,0.0,0,1.500000,0.000000,True
3,2023.0,1359475,GT,CSK,CSK,1,4,0.0,False,0.0,0,1.666667,0.000000,True
4,2023.0,1359475,GT,CSK,CSK,1,5,0.0,False,0.0,0,1.833333,0.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242545,2008.0,336040,CSK,RR,RR,20,2,0.0,False,77.0,2,20.333333,3.786885,False
242546,2008.0,336040,CSK,RR,RR,20,3,0.0,False,77.0,3,20.500000,3.756098,False
242547,2008.0,336040,CSK,RR,RR,20,4,0.0,False,77.0,3,20.666667,3.725806,False
242548,2008.0,336040,CSK,RR,RR,20,4,0.0,False,77.0,3,20.666667,3.725806,False


In [7]:
ball_by_ball_df.to_csv("sample_dataset.csv")

In [8]:
# Define features (X) and target (y)
X = ball_by_ball_df[['overs_bowled', 'wickets_lost', 'run_rate', 'powerplay']]
y = ball_by_ball_df['cumulative_runs']

# Convert powerplay from boolean to int
X['powerplay'] = X['powerplay'].astype(int)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19172\1075537488.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['powerplay'] = X['powerplay'].astype(int)


In [9]:
X_train

,overs_bowled,wickets_lost,run_rate,powerplay
199032,12.666667,2,2.605263,0
237647,17.166667,0,4.601942,0
10462,1.833333,1,0.000000,1
39475,8.500000,6,0.000000,0
137928,9.166667,4,0.763636,0
...,...,...,...,...
119879,10.833333,2,6.369231,0
103694,12.500000,2,0.480000,0
131932,6.000000,3,6.000000,1
146867,18.833333,2,4.725664,0


In [10]:
y_train

199032    33.0
237647    79.0
10462      0.0
39475      0.0
137928     7.0
          ... 
119879    69.0
103694     6.0
131932    36.0
146867    89.0
121958    21.0
Name: cumulative_runs, Length: 194040, dtype: float64

In [35]:
rf_model=RandomForestRegressor(n_estimators=200, max_depth=None, random_state=42,min_samples_leaf=1,min_samples_split=2)

In [37]:
rf_model

RandomForestRegressor(n_estimators=200, random_state=42)

In [38]:
rf_model.fit(X_train,y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [39]:
y_rf_simple_pred=rf_model.predict(X_test)

In [40]:
rf_simple_mse=mean_squared_error(y_test,y_rf_simple_pred)
rf_r2=r2_score(y_test,y_rf_simple_pred)

In [77]:
print('MSE-Score: ',rf_simple_mse)

MSE-Score:  0.15980264739229028


In [75]:
print('R2-Score: ' ,rf_r2)

R2-Score:  0.9999045513977854


In [69]:
new_data = pd.DataFrame({
    'overs_bowled': [14.2],   
    'wickets_lost': [4],     
    'run_rate': [5],        
    'powerplay': [0]         
})

In [81]:
print(new_data)

   overs_bowled  wickets_lost  run_rate  powerplay
0          14.2             4         5          0


In [56]:
predict_runs=rf_model.predict(new_data)
print(f"Predicted Cumulative Runs: {predict_runs[0]}")

Predicted Cumulative Runs: 70.825
